In [ ]:
%pylab ipympl
ifig = 0

import scipy.integrate
import sys
sys.path.append('../')

import nuclear

reactions = nuclear.network.read_network_file("./networks/cno_cycle") + nuclear.network.read_network_file("./networks/pp_chain")

ifig += 1
nuclear.plot.network_graph(reactions, ifig=ifig)

In [ ]:
rho = 150 # in [g/cm^3]
T = 1.5e7 # in [K]
particles, get_dYdt_epsilon = nuclear.network.build_network(reactions, dir="./reactions")

print(f"Loaded network using {len(particles):d} isotopes")

X_0 = nuclear.network.initalize_mass_fraction(particles)
Y_0 = nuclear.network.mass_frac_to_molar_abu(particles, X_0)

In [ ]:
lt_range = numpy.array([0, 20])
t_range = 10.0**lt_range

t, Y = nuclear.network.solve_network(rho, T, Y_0, get_dYdt_epsilon, t_range, atol=1e-9, rtol=1e-9)

In [ ]:
ifig += 1
nuclear.plot.isotope_evolution(t, Y, particles, ifig=ifig)